In [1]:
import pandas as pd
import numpy as np

# Read Data1.xlsx into a dataframe
elc_stock_df = pd.read_excel('data_files/elc_stock.xlsx')

In [2]:
elc_stock_df.columns

Index(['Plnt', 'SLoc', 'Material', 'MS Code', 'Material Description', 'L/O',
       'Product hierarchy', 'Pl', 'PDT', 'Typ',
       'Current Free Stock for Ordering OpCo', 'BUn', 'ProcType',
       'Reorder Point', 'BUn.1', 'Safety Stock', 'BUn.2', 'Min. Lot Sze',
       'BUn.3', 'SG', 'PGr', 'LS', 'SPT', 'Profl.', 'Rounding val.',
       'Language', 'Orig', 'Code number', 'Date of Last Count', 'MRPC', 'Bin',
       'MTyp', 'BUn.4', 'Pro.', 'Orig.1', 'Matl Group', 'GRT', 'Max. level',
       'BUn.5', 'Blocked', 'BUn.6', 'M', 'SPT.1', 'Replenishmt qty', 'BUn.7',
       'Reorder point', 'BUn.8', 'Order Instruction Control Type', 'MRPpr',
       'Security export control item', 'ESLoc', 'Orig.2', 'SLoc.1',
       'Standard price', 'Crcy', 'Pe', 'Total Stock', 'BUn.9', 'per',
       'Total Value', 'Crcy.1'],
      dtype='object')

In [10]:
elc_stock_df['MS Code'] = elc_stock_df['MS Code'].fillna(elc_stock_df['Material'])
elc_stock_df['BU'] = elc_stock_df['Product hierarchy'].str.slice(stop=5)
elc_stock_df_l = elc_stock_df.loc[:, ['MS Code', 'BU', 'Current Free Stock for Ordering OpCo', 'Safety Stock']]

In [11]:
# Drop rows where column MS Code is NaN
elc_stock_df_l = elc_stock_df_l.dropna(subset=['MS Code'])

In [14]:
key_bu = ['YY112', 'YY113', 'YY114', 'YY115', 'YY116', 'YY117','YY118', 'YY119', 'YY138']

elc_stock_df_l = elc_stock_df_l[elc_stock_df_l['BU'].isin(key_bu)]

In [15]:
# Write the DataFrame to an Excel file
elc_stock_df_l.to_excel('data_files/output.xlsx', index=False)